In [ ]:
! pip install -U spacy>=3.0

In [ ]:
!python -m spacy download ru_core_news_md

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('ru_core_news_md')

In [1]:
import re
import pandas as pd
from collections import Counter, OrderedDict

In [2]:
TRANSLIT_TABLE = {"дя" : "d'a",
    "тя" : "t'a",
    "ня" : "n'a",
    "де" : "d'e",
    "те" : "t'e",
    "че" : "če",
    "не" : "n'e",
    "дю" : "d'u",
    "тю" : "t'u",
    "ню" : "n'u",
    "дё" : "d'o",
    "тё" : "t'o",
    "нё" : "n'o",
    "сё" : "s'o",
    "ди" : "d'i",
    "а" : "a",
    "б" : "b",
    "в" : "w",
    "ҕ" : "γ",
    "г" : "g",
    "дь" : "d'",
    "ть" : "t'",
    "д" : "d",
    "ж" : "ž",
    "з" : "z",
    "и" : "i",
    "й" : "j",
    "ӄ" : "q",
    "к" : "k",
    "ль" : "l'",
    "л" : "l",
    "м" : "m",
    "нь" : "n'",
    "н" : "n",
    "ӈ" : "ŋ",
    "о" : "o",
    "п" : "p",
    "р" : "r",
    "с" : "s",
    "т" : "t",
    "у" : "u",
    "ф" : "f",
    "ч" : "č",
    "ц" : "c",
    "ш" : "š",
    "щ" : "š'",
    "ы" : "ɨ",
    "э" : "e",
    "ья" : "ja",
    "ью" : "ju",
    "ъе" : "je",
    "ъя" : "ja",
    "ъю" : "ju",
    "ю" : "ju",
    "я" : "ja",
    "е" : "je",
    "ъ" : "",
    "ь" : "'",
    "." : ".",
    "," : ",",
    "?" : "?",
    "!" : "!",
    "-" : "-",
    }

translit = OrderedDict(TRANSLIT_TABLE)
vowels = {"е":"\'e", "я":"\'a", "ю":"\'u", "ё":"\'o"}

In [8]:
TRANSLIT_TABLE = {"дя" : "ďa",
    "тя" : "ťa",
    "ня" : "n'a",
    "де" : "ďe",
    "те" : "ťe",
    "не" : "n'e",
    "дю" : "ďu",
    "тю" : "ťu",
    "ню" : "n'u",
    "дё" : "ďo",
    "тё" : "ťo",
    "нё" : "n'o",
    "сё" : "s'o",
    "ди" : "ďi",
    "а" : "a",
    "б" : "b",
    "в" : "w",
    "ҕ" : "γ",
    "г" : "g",
    "дь" : "ď",
    "ть" : "ť",
    "д" : "d",
    "ье" : "je",
    "е" : "je",
    "ё" : "jo",
    "ж" : "ž",
    "з" : "z",
    "и" : "i",
    "й" : "j",
    "ӄ" : "q",
    "к" : "k",
    "ль" : "l",
    "л’" : "l",
    "л" : "l",
    "м" : "m",
    "нь" : "n'",
    "н" : "n",
    "ӈ" : "ŋ",
    "о" : "o",
    "ө": "ɵ",
    "п" : "p",
    "р" : "r",
    "с" : "s",
    "т" : "t",
    "у" : "u",
    "ч" : "č",
    "ь" : "'",
    "ш" : "š",
    "щ" : "š'",
    "ы" : "ɨ",
    "э" : "e",
    "ья" : "ja",
    "ью" : "ju",
    "ю" : "ju",
    "я" : "ja",
    "." : ".",
    "," : ",",
    "?" : "?",
    "!" : "!",
    "-" : "-",
    }

translit = OrderedDict(TRANSLIT_TABLE)
vowels = {"е":"\'e", "я":"\'a", "ю":"\'u", "ё":"\'o"}

In [9]:
def transliterate(string, translit, vowels):
    pattern = re.compile(r'(?<=[врпсгклзцвбмшжӈҕӄВРПСГКЛЗЦВБМШЖӇӃҔ])[еяюё]')
    string = pattern.sub(lambda x:vowels[x.group()], string)
    for cyr, ipa in translit.items():
        string = string.replace(cyr, ipa).replace(cyr.title(), ipa.title())
    return string

In [ ]:
df = pd.read_csv('eve_base_translation.csv', delimiter=',')

In [ ]:
df['base_lat'] =0
df['POS'] = 0
for index, row in df.iterrows():
    base = row['base_new'].replace('̄', '')
    df['base_lat'].loc[index] = transliterate(base, translit, vowels)
    if row['base_new'].endswith('-'):
        df['POS'].loc[index] = 'verb'
    else:
        df['POS'].loc[index] = nlp(row['transl'].split('.')[0])[0].pos_.lower()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
df.to_csv('base_POS.csv')